# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'c:Users/marcadentsandra/Documents/Cours_EPFL/Master/Master_3/Applied_data/repo_tuto/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [3]:
from datetime import datetime
from datetime import date, time
import glob 

In [21]:
# Write your answer here
# Import Data

DATA_FOLDER='Data'
countries=pd.Series(['guinea_data','liberia_data','sl_data'])
frame = pd.DataFrame()
list_ = []
d={}
    
for c in range(3):
    print(c)
    list_=[]
    path=DATA_FOLDER+'/ebola/'+countries[c]
    allFiles = glob.glob(path + "/*.csv")
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
    frame=pd.concat(list_)
    frame=treat_data(frame)
    d[c]=frame

#frame = pd.concat(list_)

# changer pd.MultiIndex.from_tuples(clumns)
# date.split('/').reverse().concatenate(-) puis map(dates)
# mettre pays (hopitaux), description en indexe et date en colonnes

0
1
2


In [15]:
d[0].head()

,Beyla,Boffa,Conakry,Coyah,Dabola,Dalaba,Date,Description,Dinguiraye,Dubreka,...,Kouroussa,Lola,Macenta,Mzerekore,Nzerekore,Pita,Siguiri,Telimele,Totals,Yomou
0,NaN,0,5,NaN,0,NaN,2014-08-04,New cases of suspects,0,0,...,0,NaN,0,NaN,0,0,0,0,5,NaN
1,NaN,0,0,NaN,0,NaN,2014-08-04,New cases of probables,0,0,...,0,NaN,0,NaN,0,0,0,0,0,NaN
2,NaN,0,1,NaN,0,NaN,2014-08-04,New cases of confirmed,0,0,...,0,NaN,0,NaN,0,0,0,0,4,NaN
3,NaN,0,6,NaN,0,NaN,2014-08-04,Total new cases registered so far,0,0,...,0,NaN,0,NaN,0,0,0,0,9,NaN
4,NaN,0,9,NaN,0,NaN,2014-08-04,Total cases of suspects,0,0,...,0,NaN,0,NaN,0,0,0,0,11,NaN


In [33]:

d[0] = treat_data(d[0])

dfs = [d[0], d[1], d[2]]

df_combined = pd.concat(dfs,keys=['HEADER TITLE{}'.format(i) for i in range(1, len(dfs) + 1)],axis=0)

df_combined.head(100)


TypeError: 'values' is not ordered, please explicitly specify the categories order by passing in a categories argument.

In [ ]:



d = pd.DataFrame(d.items())
#d[0] = pd.MultiIndex.from_product([d.columns, ['0']])
#d[1] = pd.MultiIndex.from_product([d.columns, ['1']])
#d[2] = pd.MultiIndex.from_product([d.columns, ['2']])
dat = pd.concat([d[0],d[1],d[2]])
#dat.head()

In [11]:
datas = pd.Series(['guinea_data','liberia_data','sl_data'], index=['Guinea','Liberia','Sierra Leone'])
datas.index[1]
datas['Guinea']

'guinea_data'

In [56]:
DATA_FOLDER='Data'
#countries=pd.Series(['guinea_data','liberia_data','sl_data'])
frame = pd.DataFrame()
tot_frame = pd.DataFrame()
list_ = []
c='liberia_data'
path=DATA_FOLDER+'/ebola/'+c
allFiles = glob.glob(path + "/*.csv")
for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
        
#pd.to_numeric(list_, errors='coerce')
frame = pd.concat(list_)

In [101]:
frame.head()

,Bomi County,Bong County,Date,Gbarpolu County,Grand Bassa,Grand Cape Mount,Grand Gedeh,Grand Kru,Lofa County,Margibi County,Maryland County,Montserrado County,National,Nimba County,River Gee County,RiverCess County,Sinoe County,Unnamed: 18,Variable
0,NaN,NaN,6/16/2014,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,Specimens collected
1,NaN,NaN,6/16/2014,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Specimens pending for testing
2,NaN,NaN,6/16/2014,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,7.0,28.0,NaN,NaN,NaN,NaN,NaN,Total specimens tested
3,NaN,NaN,6/16/2014,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN,NaN,Newly reported deaths
4,NaN,NaN,6/16/2014,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,0.0,8.0,NaN,NaN,NaN,NaN,NaN,Total death/s in confirmed cases


In [104]:
df=df.dropna(how='all',axis=0)
df.index.duplicated()

array([False, False, False, ...,  True,  True,  True], dtype=bool)

In [133]:
v=['2014-6-22']
v.astype(str)
#v = pd.to_datetime(v)

AttributeError: 'list' object has no attribute 'astype'

In [134]:
df = frame

# Standardize data


df=df.rename(columns = {'date':'Date'})
df=df.rename(columns = {'variable':'Description'})
df=df.rename(columns = {'Variable':'Description'})
df=df.dropna(how='all')
#df = df.loc[df['Description'].isnull()==False]
newv=[]

if c != 'liberia_data':
    df['Date'] = pd.to_datetime(df['Date'])
else:
    for v in df['Date'].values:
        val = v.split('/')
        val = [val[2],val[0],val[1]]
        newv.append('-'.join(val))
    #for i in range(len(newv)):
        #df['Date']= df.set_value(i,'Date',newv[i])
    #df['Date'] = pd.to_datetime(df['Date'])
    new = pd.DataFrame()
    new = concat(newv)
    new.astype(str)
    df = df.drop(df['Date'])    
    df = df.assign(Date = new)    
    #df['Date'] = pd.to_datetime(df['Date'])    


df=df.replace(['-','Nr','NR'],np.nan)
df['Description'] = df['Description'].map({'new_confirmed': 'New Cases',
                                           'death_confirmed': 'New Deaths',
                                           'Total death/s in confirmed cases':'New Deaths',
                                          'New case/s (confirmed)':'New Cases', 
                                           'Total deaths in confirmed cases':'New Deaths',
                                           'New Cases of confirmed':'New Cases'})
df = df.set_index(['Date','Description'])

df = df.loc[df.index.get_level_values('Description').isnull()==False]
df.head(200)


NameError: name 'concat' is not defined

In [40]:
df = frame

# Standardize data
df=df.rename(columns = {'date':'Date'})
df=df.rename(columns = {'variable':'Description'})
df=df.rename(columns = {'Variable':'Description'})
df=df.replace(['-','Nr','NR'],np.nan)
df = df.set_index(['Date','Description'])
df = df.loc[df.index.get_level_values('Description').isnull()==False]
df = df.loc[df.index.get_level_values('Description').str.contains('New|new|Newly|Deaths|deaths|death')]
df = df.loc[df.index.get_level_values('Description').str.contains('Total|total|Totals|totals|cum|Cum|cummulative|contacts')==False]
df = df.astype(float)
Total = pd.DataFrame(df.sum(axis=1))
Total.head(100)

0
Date       Description              
2014-08-12 new_noncase           8.0
           new_suspected        20.0
           new_probable          2.0
           new_confirmed        22.0
           death_suspected      10.0
           death_probable       68.0
           death_confirmed     528.0
           etc_new_admission    23.0
           etc_new_deaths        8.0
           etc_new_discharges    9.0
2014-08-13 new_noncase          26.0
           new_suspected         6.0
           new_probable          2.0
           new_confirmed        30.0
           death_suspected      10.0
           death_probable       68.0
           death_confirmed     546.0
           etc_new_admission    13.0
           etc_new_deaths        6.0
           etc_new_discharges    9.0
2014-08-14 new_noncase          52.0
           new_suspected         0.0
           new_probable          4.0
           new_confirmed        26.0
           death_suspected      10.0
           death_probable       68.0
           death_confirmed     560.0
           etc_new_admission     5.0
           etc_new_deaths        7.0
           etc_new_discharges    1.0
...                              ...
2014-08-19 new_noncase          36.0
           new_suspected        32.0
           new_probable          6.0
           new_confirmed        18.0
           death_suspected      10.0
           death_probable       68.0
           death_confirmed     640.0
           etc_new_admission     9.0
           etc_new_deaths        6.0
           etc_new_discharges    5.0
2014-08-20 new_noncase           8.0
           new_suspected         2.0
           new_probable          0.0
           new_confirmed         8.0
           death_suspected      10.0
           death_probable       68.0
           death_confirmed     644.0
           etc_new_admission     4.0
           etc_new_deaths        2.0
           etc_new_discharges    2.0
2014-08-21 new_noncase          16.0
           new_suspected         0.0
           new_probable          0.0
           new_confirmed        18.0
           death_suspected      16.0
           death_probable       68.0
           death_confirmed     658.0
           etc_new_admission     3.0
           etc_new_deaths        7.0
           etc_new_discharges    9.0

[100 rows x 1 columns]

In [320]:
averages = pd.DataFrame()
groups = Total.groupby(Total.index.get_level_values('Description'))
averages = groups.agg(np.mean)
averages.columns = ['Daily Average in year 2014 at '+ c]
averages.head()

,Daily Average in year 2014 at sl_data
Description,
death_confirmed,1408.912621
death_probable,202.320388
death_suspected,124.631068
etc_new_admission,15.621359
etc_new_deaths,4.213592


In [32]:
def treat_data(df):
    df=df.rename(columns = {'date':'Date'})
    df=df.rename(columns = {'variable':'Description'})
    df=df.rename(columns = {'Variable':'Description'})
    df=df.replace(['-','Nr','NR'],np.nan)
    df = df.set_index(['Date','Description'])
    df = df.loc[df.index.get_level_values('Description').isnull()==False]
    
    return df

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here